In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!pip install -qU pinecone-client pinecone-datasets langchain-pinecone

In [ ]:
!pip install --quiet langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python langchain-mistralai gpt4all

In [ ]:
%pip install --upgrade --quiet  wikipedia

In [38]:
import os
import bs4
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_community.chat_models import ChatOllama
from operator import itemgetter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import pinecone
from pinecone import Pinecone, ServerlessSpec, PodSpec
from langchain_community.document_loaders import WikipediaLoader
from google.colab import userdata
import time
from langchain_community.embeddings import OllamaEmbeddings

In [6]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# In the following two lines either replace the function with 
# your API keys or save them in the secrets section in google 
# colab
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
pinecone_api_key = os.environ['PINECONE_API_KEY']
use_serverless = True

In [7]:
# Loading Wikipedia articles about Boeing, in this 
# case we load 24 Boeing related articles
docs = WikipediaLoader(query="Boeing").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [8]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

In [14]:
# We use mxbai embeddings (state of the art embedding as of 
# March 2024)
# As of March 2024, this model archives SOTA performance for 
# Bert-large sized models on the MTEB. It outperforms commercial models 
# like OpenAIs text-embedding-3-large model and matches the performance 
# of model 20x its size.
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
len(embeddings.embed_query("test query"))

1024

In [18]:
pc = Pinecone(api_key=pinecone_api_key)
if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec()
# check for and delete index if already exists
index_name = 'boeing-db'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
# create a new index
pc.create_index(
    index_name,
    dimension=1024,  # dimensionality of mxbai-embed-large
    metric='dotproduct',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [19]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [20]:
from langchain.vectorstores import Pinecone
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Pinecone.from_documents(splits, embeddings, index_name = index_name)
retriever = vectorstore.as_retriever()

In [23]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 413}},
 'total_vector_count': 413}

In [25]:
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [29]:
# LLM
local_llm = "mistral-openorca"

llm = ChatOllama(model=local_llm, temperature=0.75)

In [37]:
generate_queries_step_back = (prompt |
                              llm |
                              StrOutputParser() |
                              (lambda x: x[:-10]))
question = "Out of all boeing aircraft which one could be considered their best work"
generate_queries_step_back.invoke({"question": question})

'which Boeing aircraft is highly regarded?'

In [40]:
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
    | (lambda x: x[:-10])
)

chain.invoke({"question": question})

' The question of which Boeing aircraft is the "best" is subjective and depends on various factors, such as a person\'s preference, industry standards, and the specific needs of the user. However, some models are widely regarded as successful and innovative designs, including the Boeing 787 Dreamliner and the Boeing 747 Jumbo Jet.\n\nThe 787 Dreamliner is known for its advanced materials, fuel efficiency, and passenger comfort features. It was designed to be more environmentally friendly, with a focus on reducing emissions and noise levels. The 787 has also received praise for its large windows, mood lighting, and improved air quality systems that contribute to a better in-flight experience for passengers.\n\nThe 747 Jumbo Jet revolutionized the aviation industry upon its introduction in 1969. It was the first wide-body aircraft, allowing airlines to transport more passengers and cargo at once while also offering greater comfort with its spacious interior layouts. The 747\'s unique des